In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from PIL import Image
from torch.utils.data import DataLoader
import json
import os
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
import numpy as np
import requests

In [ ]:
# Data preprocessing and augmentation
train_transform = transforms.Compose([
    transforms.Resize((300, 300)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=60),
    transforms.RandomAffine(degrees=0, translate=(0.15, 0.15), scale=(0.7, 1.3)),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.2),
    transforms.RandomResizedCrop((300, 300), scale=(0.7, 1.0), ratio=(0.75, 1.333)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalize to ImageNet standards
])

val_transform = transforms.Compose([
    transforms.Resize((300, 300)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
# Load datasets
train_dataset = datasets.ImageFolder('train', transform=train_transform)
val_dataset = datasets.ImageFolder('valid', transform=val_transform)

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Classes
class_names = train_dataset.classes
print(class_names)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Load pre-trained EfficientNet-B3
model = models.efficientnet_b3(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

# Modify the final layer
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Sequential(
    nn.Linear(num_ftrs, 512),
    nn.BatchNorm1d(512),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(512, 128),
    nn.BatchNorm1d(128),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(128, len(class_names)) 
)

model = model.to(device)

In [ ]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()

for name, param in model.named_parameters():
    if 'classifier' in name or 'features.6' in name or 'features.7' in name:
        param.requires_grad = True
    else:
        param.requires_grad = False 

# Define optimizer with different learning rates 
optimizer = optim.Adam([
    {'params': model.classifier.parameters(), 'lr': 0.001}, 
    {'params': [param for name, param in model.named_parameters() if 'classifier' not in name and param.requires_grad], 'lr': 0.0001} 
], lr=0.0001) 

# Add a learning rate scheduler 
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)


In [ ]:
# Training loop
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler=None, num_epochs=10, history_file='history.json'):
    history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []}
    best_val_loss = float('inf')
    early_stop_patience = 5 
    patience_counter = 0 

    for epoch in range(num_epochs):
        model.train()
        train_loss, correct_train = 0, 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            correct_train += (outputs.argmax(1) == labels).sum().item()

        if scheduler:
            scheduler.step()

        val_loss, correct_val = 0, 0
        model.eval()
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                correct_val += (outputs.argmax(1) == labels).sum().item()

        # Calculate accuracy and average loss
        train_loss /= len(train_loader) # Calculate average train loss per batch
        val_loss /= len(val_loader)   # Calculate average val loss per batch
        train_acc = correct_train / len(train_loader.dataset)
        val_acc = correct_val / len(val_loader.dataset)

        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['train_acc'].append(train_acc)
        history['val_acc'].append(val_acc)

        print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

        # Early stopping check
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            # Optionally save the best model here
            torch.save(model.state_dict(), 'best_model.pth')
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= early_stop_patience:
                print("Early stopping triggered.")
                break # Stop training loop

    # Load the best model state if saved
    if os.path.exists('best_model.pth'):
        model.load_state_dict(torch.load('best_model.pth'))
        print("Loaded best model state.")

    # Save final history to JSON file
    with open(history_file, 'w') as f:
        json.dump(history, f, indent=4)

    return history

# Train the model
# Pass the scheduler to the training function
history = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler=scheduler, num_epochs=10) # Increased epochs as example

In [ ]:
# Save the final trained model 
torch.save(model.state_dict(), 'plant_disease_model.pth')

In [ ]:
# Model Evaluation after training 
print("\nEvaluation on Validation Set:")
model.eval()
val_labels, val_preds = [], []
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        val_preds.extend(outputs.argmax(1).cpu().numpy())
        val_labels.extend(labels.cpu().numpy())

print("Confusion Matrix:")
print(confusion_matrix(val_labels, val_preds))
print("\nClassification Report:")
 
# Ensure class_names is available and correct for target_names
print(classification_report(val_labels, val_preds, target_names=class_names))

In [ ]:
# Calculate ROC AUC, ensure 
try:
    val_probs = []
    with torch.no_grad():
        for inputs, labels in val_loader:
          inputs = inputs.to(device)
          outputs = model(inputs)           
          val_probs.extend(torch.softmax(outputs, dim=1).cpu().numpy())
          val_probs = np.array(val_probs)
          
    # For multi-class, you might need to calculate per-class AUC or macro/weighted average
    
    if len(class_names) > 2:
        roc_auc = roc_auc_score(val_labels, val_probs, multi_class='ovo', average='macro') 
        print(f"\nMacro Average ROC AUC: {roc_auc:.4f}")
except ValueError as e:
    print(f"Could not calculate ROC AUC: {e}")
except Exception as e:
     print(f"An error occurred during ROC AUC calculation: {e}") 